# Ejercicio 3: \"Conjunto de test\" (obligatorio)

Reportar los resultados en el conjunto de test.

Entrenar los resultados en toda la data de desarrollo (incluyendo x_valid), y volver a evaluar el test. Esto no se debería hacer, por qué?

En la práctica, no obstante es común entrenar la configuración elegida en toda la data de desarrollo y luego evaluar en el test. Notar que esto no supone un conflicto siempre y cuando evaluemos en test UNA SOLA VEZ (no vale elegir entre el modelo entrenado en toda la data de desarrollo.
Que ventajas tiene hacer esto?

# Solucion

## 1.

### Creacion de los modelos y evaluacíon en el conjunto de `train`

In [3]:
import numpy as np
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/geronimoforconi/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/geronimoforconi/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/geronimoforconi/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/geronimoforconi/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/geronimoforconi/nltk_data...
[nltk_data]    |   Package averaged_perce

True

In [2]:
from gensim.test.utils import datapath, get_tmpfile

embeddings_path = '../models/glove.6B/glove.6B.300d.txt'

glove_file = datapath(embeddings_path)
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")

glove2word2vec(embeddings_path, word2vec_glove_file)

model_en = KeyedVectors.load_word2vec_format(word2vec_glove_file)

/var/folders/ck/qx4468vx3sx6bf4nt0smfg2c0000gn/T/ipykernel_23980/796777347.py:8: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(embeddings_path, word2vec_glove_file)


In [4]:
import os.path 
import glob
import pandas as pd



def get_text(file_path):
    with open(file_path, 'r') as f:
        return '\n'.join(f.readlines())

def load_dataset(is_train, limit):
    dataset_path = '../data/aclImdb/'
    cls = ['pos', 'neg']

    data_path = os.path.join(dataset_path, 'train' if is_train else 'test')
    data = []
    limit_per_class = limit//2
    for c in cls:
        class_path = os.path.join(data_path,c)
        regex_glob = os.path.join(class_path,"*.txt")
        for i, file_path in enumerate(glob.glob(regex_glob)):
            if i == limit_per_class:
                break
            data.append((get_text(file_path),c))
    return pd.DataFrame(data=data,columns=['text', 'class'])
            
        
df = load_dataset(is_train=True, limit=100000).sample(frac=1)
df.head(10)

,text,class
4539,The first film ever made. Workers streaming fr...,pos
23892,This movie looked as if it might be good at th...,neg
15915,This film is justly famous as one of the most ...,neg
1758,"After Racism, Rural exodus -also known as migr...",pos
8840,"""The Gingerbread Man is the first thriller I'v...",pos
20538,"Motocrossed was fun, but it wasn't that great....",neg
21791,I have to admit I laughed a few times during t...,neg
10915,Somewhat funny and well-paced action thriller ...,pos
1030,A young ( only 21 ) director with great talent...,pos
4892,By some happy coincidence the same year that J...,pos


In [5]:
def get_text_embedding(text):
    words = nltk.word_tokenize(text.lower())
    l = [model_en[w] for w in words if w in model_en]
    if not l:
        return np.zeros((300,))
    return np.array(l).mean(axis=0)
#get_text_embedding(text).shape 

Llego el momento de calcular los embeddings de todo el texto:

In [6]:
import tqdm

def get_df_embeddings(df):
    embs = []
    for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
        embs.append(get_text_embedding(row['text']))
    embs=np.array(embs)
    return embs

In [7]:
X = get_df_embeddings(df)

100%|██████████| 25000/25000 [00:18<00:00, 1347.58it/s]


Y calcularemos las targets (variable dependiente):

In [ ]:
y = (df['class']=='pos').astype(int)

Nos separaremos un split de validación:

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((20000, 300), (5000, 300), (20000,), (5000,))

Y finalmente entrenaremos y evaluaremos algunos clasificadores simples: uno basado en KNN y otro basado en RandomForest.

In [10]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=11)
knn_clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=11)

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

RandomForestClassifier()

In [12]:
pred_rf = rf_clf.predict(X_valid)
pred_knn = knn_clf.predict(X_valid)

probs_rf = rf_clf.predict_proba(X_valid)
probs_knn = knn_clf.predict_proba(X_valid)
ensamble_pred = ((probs_rf[:,1]+probs_knn[:,1])/2.0)>0.5

pred_rf.shape, pred_knn.shape, ensamble_pred.shape

((5000,), (5000,), (5000,))

In [13]:
((probs_rf[:,1]>0.5) == pred_rf).all()

True

In [14]:
((probs_knn[:,1]>0.5) == pred_knn).all()

True

In [15]:
from sklearn.metrics import accuracy_score

print(f'Random Forest acc: {accuracy_score(y_valid,pred_rf)}')
print(f'KNN acc: {accuracy_score(y_valid,pred_knn)}')
print(f'Ensamble acc: {accuracy_score(y_valid,ensamble_pred)}')

Random Forest acc: 0.783
KNN acc: 0.7276
Ensamble acc: 0.7672


### Ejecucion de los modelos con los conjuntos de `test`

In [16]:
df_tst = load_dataset(is_train=False, limit=100000).sample(frac=1)
df_tst.head(10)

,text,class
7255,I saw MESSIAH 2 a few months ago and didn`t ge...,pos
14638,"Danny De Vito shows us here he is definitely, ...",neg
8256,o m g!!! did you ever think they would make a ...,pos
8324,Aaron Sorking raises the same questions as Sha...,pos
8770,"""Closet Land"" was sponsored by Amnesty Interna...",pos
9096,The notion of marital fidelity portrayed in th...,pos
24556,This is by far the worst movie I have ever see...,neg
18951,I must say I was surprised to find several pos...,neg
20707,This movie is stupid and i hate it!!! i turned...,neg
1630,I watched Asterix and Obelix in Operation Cleo...,pos


In [17]:
X_tst = get_df_embeddings(df_tst)

100%|██████████| 25000/25000 [00:17<00:00, 1390.54it/s]


In [18]:
y_tst = (df_tst['class']=='pos').astype(int)

En esta etapa no dividimos en traint y val el dataset de test ya que suponemos que los modelos fueron creados y sus hiperparametros ajustados. Asi que el proximo paso seria evaluar el dataset de test y comparar los resultados.

In [19]:
pred_rf_tst = rf_clf.predict(X_tst)
pred_knn_tst = knn_clf.predict(X_tst)

probs_rf_tst = rf_clf.predict_proba(X_tst)
probs_knn_tst = knn_clf.predict_proba(X_tst)
ensamble_pred_tst = ((probs_rf_tst[:,1]+probs_knn_tst[:,1])/2.0)>0.5

pred_rf_tst.shape, pred_knn_tst.shape, ensamble_pred_tst.shape

((25000,), (25000,), (25000,))

In [21]:
print(f'Random Forest acc: {accuracy_score(y_tst,pred_rf_tst)}')
print(f'KNN acc: {accuracy_score(y_tst,pred_knn_tst)}')
print(f'Ensamble acc: {accuracy_score(y_tst,ensamble_pred_tst)}')

Random Forest acc: 0.76836
KNN acc: 0.71732
Ensamble acc: 0.75456


### Comparacion de resultados

| Modelo  | Train  | Test   |
|------------|------------|------------|
| Random Forest acc | 0.783 | 0.76836 |
| KNN acc | 0.7276 | 0.71732 |
| Ensamble acc | 0.7672 | 0.75456 |

<strong> Conclusion: </strong>  Para todos los modelos el accuracy en test es levemente mas bajo que en train, podemos considerar que los modelos si bien no tienen un acurracy alto no tiene un sobreajuste y es de esperar que los modelos evaluados con datos de test de un valor mas bajo que los probados con train, principalmente porque son datos nuevos los cuales no fueron utilizados para entrenar.

## 2.

Entrenar los resultados en toda la data de desarrollo (incluyendo x_valid), y volver a evaluar el test. Esto no se debería hacer, por qué?

<strong> Porque no tiene sentido evaluar los modelos con datos que se utilizaron para entrenar el modelo.

Estos datos ya fueron evaluados e incluso se comparo su entrada con su salida, esto hace el entrenamiento. </strong>

## 3.

En la práctica, no obstante es común entrenar la configuración elegida en toda la data de desarrollo y luego evaluar en el test. Notar que esto no supone un conflicto siempre y cuando evaluemos en test UNA SOLA VEZ (no vale elegir entre el modelo entrenado en toda la data de desarrollo.
Que ventajas tiene hacer esto?

<strong> Evitas un sobreajuste del modelo, si evaluas muchas veces el mismo conjunto y buscas configurar los hiperparametros del modelo para mejorar las metricas de rendimiento que estes utilizando sin darte cuenta esta haciendo que el modelo responda unicamente a tu set de datos.
Por otro lado, al evaluar una sola vez los datos de test ayuda a simular como funcionaria el modelo con datos desconocidos simulando una puesta en producción. </strong>